In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
df = pd.read_csv('Team Assignment 1 Data.csv', delimiter = ';')


In [3]:
df['Age'] = df['Age'].str.replace(',', '.').astype(float)

df['Age'] = df['Age'].astype(int)


In [4]:
df.head()

Review_ID  Number_Words_Review ZNumber_Words_Review  Prod_Desig  \
0          1                   34         -0,178828858           1   
1          2                   32         -0,225796385           1   
2          3                   54          0,290846409           1   
3          4                   20         -0,507601545           1   
4          5                   12         -0,695471652           1   

   Prod_Design_positive  Prod_Design_negative  Prod_Technical  \
0                     1                     0               1   
1                     1                     0               1   
2                     0                     1               1   
3                     1                     0               1   
4                     1                     0               1   

   Prod_Technical_positive  Prod_Technical_negative  Prod_Price  ...  \
0                        1                        0           0  ...   
1                        1                        0           1  ...   
2                        0                        1           1  ...   
3                        1                        0           0  ...   
4                        1                        0           0  ...   

   Serv_Delivery_positive  Serv_Delivery_negative  Category  Country  Gender  \
0                       0                       0         0        1       0   
1                       0                       0         0        1       0   
2                       0                       1         0        1       0   
3                       0                       0         0        1       0   
4                       0                       0         0        1       0   

   Age  Sentiment  Rating_Score  Purchase  Number_of_Purchases  
0   24          4             5         1                    1  
1   38          5             4         0                    0  
2   46          2             2         0                    0  
3   31          5             5         1                    2  
4   25          3             5         0                    0  

[5 rows x 23 columns]

We don't consider 'Number of purchases' and 'purchase' from the start because they are done after the review so they'll likely not effect the review score
We use only positive and negative topics to avoid redundancy

In [5]:
columns = ['Prod_Design_positive','Prod_Design_negative','Prod_Technical_positive','Prod_Technical_negative','Prod_Price_positive','Prod_Price_negative','Serv_Delivery_positive','Serv_Delivery_negative','Sentiment','Number_Words_Review','Gender','Age']
#ZNumber_Words_Review;Category;Country;Gender;Age;Sentiment;Rating_Score;Purchase;Number_of_Purchases
X = df[columns]
X.head()
Y = df['Rating_Score']

In [6]:
import pandas as pd
from sklearn import linear_model
import statsmodels.api as sm

In [13]:
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
predictions = model.predict(X)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:           Rating_Score   R-squared:                       0.678
Model:                            OLS   Adj. R-squared:                  0.671
Method:                 Least Squares   F-statistic:                     95.01
Date:                Tue, 19 Mar 2024   Prob (F-statistic):          8.50e-135
Time:                        23:32:05   Log-Likelihood:                -639.55
No. Observations:                 600   AIC:                             1307.
Df Residuals:                     586   BIC:                             1369.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

Based on **t-statistic** and **P-value** we notice that:

- product design negative
- product price 
- service delivery positive
- number words review
- gender
- age

are not statistically significant. So we'll try another regression without them to see changes in $\R^2$

In [22]:
columns = ['Prod_Design_positive','Prod_Technical_positive','Prod_Technical_negative','Serv_Delivery_negative','Sentiment']
#ZNumber_Words_Review;Category;Country;Gender;Age;Sentiment;Rating_Score;Purchase;Number_of_Purchases
X = df[columns]
X.head()
Y = df['Rating_Score']

In [11]:
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
predictions = model.predict(X)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:           Rating_Score   R-squared:                       0.677
Model:                            OLS   Adj. R-squared:                  0.674
Method:                 Least Squares   F-statistic:                     249.2
Date:                Tue, 19 Mar 2024   Prob (F-statistic):          3.16e-143
Time:                        23:23:39   Log-Likelihood:                -640.52
No. Observations:                 600   AIC:                             1293.
Df Residuals:                     594   BIC:                             1319.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [19]:
plt.scatter(predictions, Y, color='blue', label='Real Data')
plt.plot(predictions, predictions, color='red', label='Linear Regression')
plt.xlabel('Predicted Rating')
plt.ylabel('Actual Rating')
plt.title('Linear Regression vs Real Data')
plt.legend()
plt.show()


But because we have as a target a categorical ordinal data, we think that ***Ordered Logit model*** 
would fit better the data

In [28]:
X = sm.add_constant(X)
model = sm.MNLogit(Y, X).fit()
predictions = model.predict(X)
print_model = model.summary()
print(print_model)


Optimization terminated successfully.
         Current function value: 0.713283
         Iterations 9
                          MNLogit Regression Results                          
Dep. Variable:           Rating_Score   No. Observations:                  600
Model:                        MNLogit   Df Residuals:                      576
Method:                           MLE   Df Model:                           20
Date:                Wed, 20 Mar 2024   Pseudo R-squ.:                  0.3868
Time:                        00:22:09   Log-Likelihood:                -427.97
converged:                       True   LL-Null:                       -697.93
Covariance Type:            nonrobust   LLR p-value:                1.247e-101
         Rating_Score=2       coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                      -0.8091      0.924     -0.875      0.381      -2.621   